## Test genesis data

JSON files fetched from commit [`9bf4599`](https://github.com/lukso-network/network-configs/commit/9bf459922eef6237434d61428f94d46ece2e3cb0).

In [1]:
import json

In [2]:
with open('../data/genesis_deposit_data.json', 'r') as f:
  genesis_deposit_data = json.load(f)

In [17]:
NUMBER_OF_DEPOSITS = 10336
len(genesis_deposit_data)

10336

In [9]:
unique_pubkeys = set()
for deposit in genesis_deposit_data:
    if deposit['pubkey'] in unique_fields:
        print(f"{deposit['pubkey']} is not unique.")
    else:
        unique_pubkeys.add(deposit['pubkey'])

print("All pubkey are unique.")

All pubkey are unique.


We have 10336 unique pubkeys as expected. We could also do a more precise check for each of these keys on: `isPubkeyRegistered`.

Let's check if each of these pubkey is each genesis file.

In [23]:
genesis_deposit_data

[{'deposit_data_root': '5d8288d8a56b804497896d4fad764556bd55b07a507ee14d2262a507caab561a',
  'pubkey': 'b01705e35f9a7cbdd67d5070cd3f828cc297fb4faabe419d584a24fd59c08eb313f1ea7c0946c33548477ab849be3c7e',
  'signature': '8437a80fac78e96d8597870f87731fe21903e52e76e3c69760376c86fe9cd4d081b5139d7b52fbf3192e45ee7f4fe6340946080baa03ea5322b495912b547e1ab95b522900123c10fe6cb022440ca443863dcd2c8450898a6892ca63b0024b80',
  'amount': 32000000000,
  'version': 1,
  'withdrawal_credentials': '0100000000000000000000003bb4893b1ef4b9e2c2db0ec26ac3f078e5fcbbf8'},
 {'deposit_data_root': '0703a38769fd750865bc275433f2cb6005a6b5273f7376b634ab3451f9254c50',
  'pubkey': '80637672dbd3eaaf192c9c1b9d742d5dc5f6d853db095f7556266a38febf021998fc58ab3f2012449fae1db693f4fe5a',
  'signature': 'afe27cd35850413e1e3f66749965ad9ec69704ee6208654b6d3b2399ac618f01ee716c51d21c3b8925142a4b6968a26213ba10993788bb556408fb8aeb727c3f99f69962b5de641e447d3eb9e826d5a6328608ecc903f4b6f71718dc134831fe',
  'amount': 32000000000,
  'versio

In [25]:
pubkey_to_withdrawal_credentials = {}
for deposit in genesis_deposit_data:
    pubkey_to_withdrawal_credentials['0x%s' % deposit['pubkey']] = '0x%s' % deposit['withdrawal_credentials']

In [38]:
genesis = {}
supplies = ['35', '42', '100']

for supply in supplies:
    with open('../data/genesis_%s_ssz.json' % supply, 'r') as f:
        genesis[supply] = json.load(f)

In [43]:
def verify_validator(validator):
    if validator['slashed'] != False:
        return False
    if validator['activation_eligibility_epoch'] != "0":
        return False
    if validator['activation_epoch'] != "0":
        return False
    if validator['effective_balance'] != "32000000000": # 32 gwei (9 zeros)
        return False
    if validator['pubkey'] not in pubkey_to_withdrawal_credentials:
        return False
    if validator['withdrawal_credentials'] != pubkey_to_withdrawal_credentials[validator['pubkey']]:
        return False
    
    return True
    
"""Check if all pubkeys are correct"""
def check_committee(pubkeys):
    if len(pubkeys) != NUMBER_OF_DEPOSITS:
        return False
    
    for pubkey in pubkeys:
        if pubkey not in pubkey_to_withdrawal_credentials:
            return False
    
    return True

for supply in supplies:
    validators = genesis[supply]['validators']
    if len(validators) != NUMBER_OF_DEPOSITS:
        print('Error in %s' % supply)
        
    for validator in validators:
        if not verify_validator(validator):
            print('Error')
    
    if len(genesis[supply]['inactivity_scores']) != NUMBER_OF_DEPOSITS:
        print('Error')
    
    if check_committee(genesis[supply]['current_sync_committee']['pubkeys']):
        print('current_sync_committee error')
    
    if check_committee(genesis[supply]['next_sync_committee']['pubkeys']):
        print('next_sync_committee error')
    
    print('✅ Supply file %s is valid' % supply)

✅ Supply file 35 is valid
✅ Supply file 42 is valid
✅ Supply file 100 is valid


✅ Each supply file contains all the validators, each of them are valid.